# Introduction
I used a modification of [Guido Zuidhofs Notebook](https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial) to preprocess the data and only keep the segmented segmented_lungs_fill mask. 

You can play around with the variable IMG_SIZE and SAMPLE_RATE to change the output of the preprocessing. 

In [2]:
%matplotlib inline

import os

import dicom
from skimage import measure
import scipy.ndimage

import numpy as np
import pandas as pd
import cv2
from matplotlib import pylab as plt
from scipy import signal

from tqdm import tqdm_notebook

In [ ]:
IMG_SIZE = 200
SAMPLE_RATE = 170

# Load the scans in given folder path

In [3]:
INPUT_FOLDER = '/media/eric/My Book/Eric/stage1/'
SAVE_FOLDER = '/media/eric/My Book/Eric/preprocess_200/'
LABEL_FOLDER = '/media/eric/My Book/Eric/stage1_labels.csv'

patients = os.listdir(INPUT_FOLDER)
patients.sort()

In [4]:
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s, force=True) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

# Preprocessing Methods

In [5]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [6]:
def resize(image):
    new_patient = []
    for each_slice in patient:
        
        new_img = cv2.resize(np.array(each_slice),(IMG_SIZE, IMG_SIZE))
        new_patient.append(new_img)
    return np.array(new_patient)

In [13]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None
    
def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

# Generate Label Data

In [7]:
labels = pd.read_csv(open(LABEL_FOLDER, "rb"), quotechar=',', skipinitialspace=True)
labels = np.array(labels)
np.save(SAVE_FOLDER + 'labels', labels)

# Generate CT Input Data
Iterate through all data and apply above preprocessing methods

In [7]:
def show_output(patient, patient_id):
    patient = patient[50,:,:]
    patient = patient.copy()
    cancer = labels[np.argwhere(labels[:, 0]==patient_id), 1][0][0]
    if cancer:
        cv2.putText(patient, 'Label: Cancer', (50, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1, lineType=cv2.LINE_AA)
    else:
        cv2.putText(patient, 'Label: No Cancer', (50, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1, lineType=cv2.LINE_AA)
    cv2.imshow('frame', patient)
    cv2.waitKey(10)

In [14]:
not_processed = 0

for patient in tqdm_notebook(patients):
    patient_id = patient
    file_name = SAVE_FOLDER + patient + '.npy'
    if not os.path.isfile(file_name):
        try:
            patient = load_scan(INPUT_FOLDER + patient)
            patient = get_pixels_hu(patient)
            patient = resize(patient)
            segmented_lung = segment_lung_mask(patient)
            patient = signal.resample(segmented_lung, SAMPLE_RATE, axis=0)
            #show_output(patient, patient_id)
            np.save(file_name, patient)
        except:
            not_processed = not_processed + 1
    
print ('%d patients were not processed' % (not_processed))


1 patients were not processed
